In [1]:
!pip install transformers datasets evaluate seqeval wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=eb8238d5bcee87486a29bf84daa371a995e41543f0b38823f

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import os
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/sc2/total_ner_data.csv',index_col=0)

In [5]:
data.shape

(688672, 2)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 688672 entries, 0 to 688671
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   tokens    688672 non-null  object
 1   ner_tags  688672 non-null  object
dtypes: object(2)
memory usage: 15.8+ MB


In [7]:
data = data.drop_duplicates()

In [8]:
data.shape

(633179, 2)

In [9]:
from ast import literal_eval
data['tokens'] = data['tokens'].apply(lambda x: literal_eval(x))

In [10]:
data['ner_tags'] = data['ner_tags'].apply(lambda x: literal_eval(x))

In [11]:
data['set_ner'] = data['ner_tags'].apply(lambda x: str(set(x)))

In [12]:
data.drop(data[(data['set_ner']=='set()')].index,inplace=True)

In [13]:
data.drop(data[(data['set_ner']=="{'O'}")].index,inplace=True)

In [14]:
data['set_ner'].value_counts()

{'PS_NAME', 'O'}                                                                  93933
{'CV_RELATION', 'O'}                                                              22175
{'TMM_DISEASE', 'O'}                                                               4580
{'CV_RELATION', 'PS_NAME', 'O'}                                                    3038
{'O', 'TM_CELL_TISSUE_ORGAN'}                                                      2673
                                                                                  ...  
{'OGG_MEDICINE', 'PS_NAME', 'O', 'TR_MEDICINE'}                                       1
{'OGG_MEDICINE', 'O', 'TMM_DRUG', 'FD_MEDICINE', 'CV_RELATION', 'TMM_DISEASE'}        1
{'TR_MEDICINE', 'O', 'TMM_DRUG', 'TM_CELL_TISSUE_ORGAN', 'TMM_DISEASE'}               1
{'TMM_DRUG', 'OGG_MEDICINE', 'PS_NAME', 'O'}                                          1
{'CV_RELATION', 'TR_MEDICINE', 'O', 'TM_CELL_TISSUE_ORGAN'}                           1
Name: set_ner, Length: 95, dtype

In [ ]:
#data.drop(data[(data['set_ner']=="{'PS_NAME', 'O'}")].index,inplace=True)

In [16]:
data = data.reset_index()

In [17]:
tag_list = ['O', 'PS_NAME', 'FD_MEDICINE', 'TR_MEDICINE', 'OGG_MEDICINE', 'TMM_DISEASE',
            'TM_CELL_TISSUE_ORGAN', 'TMM_DRUG', 'CV_RELATION']

In [18]:
def chan_index(x):
  n_index= []
  for i in x:
    tag_index = tag_list.index(i)
    n_index.append(tag_index)
  return n_index

In [19]:
data['ner_index'] = data['ner_tags'].apply(lambda x: chan_index(x))

In [20]:
data['ner_index']

0                                              [0, 0, 0, 1]
1                                     [0, 0, 0, 1, 0, 0, 0]
2         [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3                                     [0, 0, 0, 1, 0, 0, 0]
4         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ...
                                ...                        
132088        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
132089                          [1, 0, 1, 0, 1, 0, 0, 0, 0]
132090                                         [0, 8, 0, 0]
132091                                      [0, 8, 0, 0, 0]
132092                                      [0, 0, 0, 8, 0]
Name: ner_index, Length: 132093, dtype: object

In [27]:
data

,level_0,index,tokens,ner_tags,set_ner,ner_index
0,0,12,"[글, │, 사진, 맑고흐림]","[O, O, O, PS_NAME]","{'PS_NAME', 'O'}","[0, 0, 0, 1]"
1,1,13,"[Copytight, ⓒ, 2019, 맑고흐림, All, rights, reserv...","[O, O, O, PS_NAME, O, O, O]","{'PS_NAME', 'O'}","[0, 0, 0, 1, 0, 0, 0]"
2,2,15,"[오늘, 오전, 예방접종, 하러, 라온이, 를, 데리고, 동물병원을, 갔다가, 요녀...","[O, O, TR_MEDICINE, O, O, O, O, O, O, O, O, O,...","{'TR_MEDICINE', 'O'}","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,22,"[Copyright, ⓒ, 2019, 맑고흐림, All, rights, reserv...","[O, O, O, PS_NAME, O, O, O]","{'PS_NAME', 'O'}","[0, 0, 0, 1, 0, 0, 0]"
4,4,43,"[나는, 천성적으로, 선생, 을, 할, 성격이, 되지, 못하고,, 무엇보다, 교사,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, TMM_DI...","{'TMM_DISEASE', 'O'}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ..."
...,...,...,...,...,...,...
132088,132088,688641,"[우리팀, 성지환, 동준이, 형, 오랜시간, 연습하느라고생했고!, 사회도보게해주시고...","[O, PS_NAME, PS_NAME, O, O, O, O, O, O, O, PS_...","{'PS_NAME', 'O'}","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
132089,132089,688651,"[로맨틱펀치, ,, 볼빨간사춘기, ,, 이진아, 님, 등등, 완전, 멋졌어요ㅠㅠ]","[PS_NAME, O, PS_NAME, O, PS_NAME, O, O, O, O]","{'PS_NAME', 'O'}","[1, 0, 1, 0, 1, 0, 0, 0, 0]"
132090,132090,688668,"[우리, 가족, 행사, ㅋ]","[O, CV_RELATION, O, O]","{'CV_RELATION', 'O'}","[0, 8, 0, 0]"
132091,132091,688669,"[시댁은, 식구, 가많아, 늘똘똘, 뭉치신다]","[O, CV_RELATION, O, O, O]","{'CV_RELATION', 'O'}","[0, 8, 0, 0, 0]"


In [33]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.3)
dataset = dataset.remove_columns(['level_0','index','set_ner'])

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_index'],
        num_rows: 92465
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_index'],
        num_rows: 39628
    })
})

In [49]:
from transformers import AutoTokenizer
from datasets import load_dataset
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True,padding=True,truncation=True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_index"][k]
    i = -1
    adjusted_label_ids = []
   
    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = tag_list[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/vocab.txt
loading file tokenizer.json from cache at /root/.cache/hu

Map:   0%|          | 0/92465 [00:00<?, ? examples/s]

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
465


Map:   0%|          | 0/39628 [00:00<?, ? examples/s]

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
628


In [67]:
tokenized_dataset['train']['tokens']

[['@tuiix_de', 'ㅠㅜ참아'],
 ['우리', '말티쎄빠뜨'],
 ['재민이', '생일이진짜얼마안남았어요', '모두가사랑하는', '재민이', '의행복한생일'],
 ['부모', '님', '카테고리안에', '많은', '샘플영상들을', '쫘악', '모두', '봤어요~'],
 ['뿌', '님', '덕분에', '울최', '도', '냠냠'],
 ['홍철', '없는', '홍철', '팀'],
 ['사촌동생', '들', '정말', '귀엽지만...귀엽긴', '하지만'],
 ['우지', '야', 'ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ'],
 ['다정이', '가', '마을모임', '때', '가져온'],
 ['혜영', '언니랑', '애들이랑', '김포', '#', '김포'],
 ['사룽해여..,,', '대장', '..,,🖤🧡💜💚❤️💕💓💗💖💘💛💙💝💟', '#', '박효신', '#', '그날'],
 ['테이', '아직도노래잘하네'],
 ['7월', '의', '지원이', '네ㅎㅎㅎ'],
 ['#이삐#사랑해#', '남매', '맘', '#', '남매', '#데일리'],
 ['울', '체리마루', '제가감히대충그릴수잇을까싶엇지만대충그려서죄송합니다', '서수진', '너무예뻐용'],
 ['사실',
  '가족',
  '이랑',
  '가면',
  '가족',
  '이',
  '칼국수',
  '는',
  '먹고,',
  '나는',
  '거의',
  '볶음밥',
  '때문에',
  '가는듯'],
 ['동생', '이', '올해', '여름부터'],
 ['허벅지', '가', '아프다며...ㅋㅋ', '양말도', '못벗는거예요..얼릉', '씻어야', '하는데..', '감기', '걸릴까봐'],
 ['최순실 게이트',
  '가',
  '터진',
  '직후엔',
  '박근혜',
  '대통령',
  '의',
  '거취,',
  '즉',
  '면책이냐',
  '하야냐',
  '탄핵이냐를',
  '걸고',
  '베팅이',
  '벌어졌고',
  '최순실',
  '씨가',
  '구속된',
  '뒤엔',
  '최순실',
  '씨의',

In [50]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [51]:
import evaluate

seqeval = evaluate.load("seqeval")

In [52]:
import wandb
os.environ["WANDB_API_KEY"]="f0b84796d34c3be4609a1b755caf0e575514e731"
os.environ["WANDB_ENTITY"]="keonju2"
os.environ["WANDB_PROJECT"]="finetune_bert_ner"

In [53]:
import numpy as np

labels = [tag_list[i] for i in data[f"ner_index"][0]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tag_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tag_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [54]:
id2label = {
    0: "O",
    1: "PS_NAME",
    2: "FD_MEDICINE",
    3: "TR_MEDICINE",
    4: "OGG_MEDICINE",
    5: "TMM_DISEASE",
    6: "TM_CELL_TISSUE_ORGAN",
    7: "TMM_DRUG",
    8: "CV_RELATION"
}
label2id = {
    "O": 0,
    "PS_NAME": 1,
    "FD_MEDICINE": 2,
    "TR_MEDICINE": 3,
    "OGG_MEDICINE": 4,
    "TMM_DISEASE": 5,
    "TM_CELL_TISSUE_ORGAN": 6,
    "TMM_DRUG": 7,
    "CV_RELATION":8,
}

In [55]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base", num_labels=9, id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "PS_NAME",
    "2": "FD_MEDICINE",
    "3": "TR_MEDICINE",
    "4": "OGG_MEDICINE",
    "5": "TMM_DISEASE",
    "6": "TM_CELL_TISSUE_ORGAN",
    "7": "TMM_DRUG",
    "8": "CV_RELATION"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "CV_RELATION": 8,
    "FD_MEDICINE": 2,
    "O": 0,
    "OGG_MEDICINE": 4,
    "PS_NAME": 1,
    "TMM_DISEASE": 5,
    "TMM_DRUG": 7,
    "TM_CELL_TISSUE_ORGAN": 6,
    "TR_MEDICINE": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": 

In [56]:
training_args = TrainingArguments(
    output_dir="korean_disease_ner",
    learning_rate=2e-5,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    evaluation_strategy="epoch",
    run_name = "ep_10",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
wandb.finish()

PyTorch: setting up devices
/content/korean_disease_ner is already a clone of https://huggingface.co/keonju/korean_disease_ner. Make sure you pull the latest changes with `repo.git_pull()`.
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_index, ner_tags, tokens. If ner_index, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 92465
  Num Epochs = 10
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Accumulati

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.100000,0.046072,0.932947,0.940132,0.936526,0.985044
2,0.034600,0.043311,0.936695,0.949956,0.943279,0.986431
3,0.021100,0.048194,0.943785,0.949291,0.946530,0.987056
4,0.013000,0.053179,0.944939,0.950128,0.947526,0.986932
5,0.009100,0.058447,0.942995,0.954868,0.948894,0.987196
6,0.006300,0.067468,0.949690,0.950342,0.950016,0.987448
7,0.004400,0.066000,0.946714,0.954289,0.950487,0.987621
8,0.003200,0.075237,0.942873,0.956262,0.949521,0.987298
9,0.002500,0.076572,0.946347,0.956091,0.951194,0.987821
10,0.001900,0.079177,0.947756,0.955319,0.951522,0.987937


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_index, ner_tags, tokens. If ner_index, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 39628
  Batch size = 50
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PS_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: TMM_DISEASE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CV_RELATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_

eval/accuracy,▁▄▆▆▆▇▇▆██
eval/f1,▁▄▆▆▇▇█▇██
eval/loss,▂▁▂▃▄▆▅▇▇█
eval/precision,▁▃▆▆▅█▇▅▇▇
eval/recall,▁▅▅▅▇▅▇███
eval/runtime,▃▄▃▃▁▅▄▄▆█
eval/samples_per_second,▆▅▆▆█▄▅▅▃▁
eval/steps_per_second,▆▅▆▆█▃▅▅▃▁
train/epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇███
train/learning_rate,██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁


In [57]:
trainer.push_to_hub()

Saving model checkpoint to korean_disease_ner
Configuration saved in korean_disease_ner/config.json
Model weights saved in korean_disease_ner/pytorch_model.bin
tokenizer config file saved in korean_disease_ner/tokenizer_config.json
Special tokens file saved in korean_disease_ner/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/420M [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/keonju/korean_disease_ner
   d9b0973..0f5dce5  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/keonju/korean_disease_ner
   d9b0973..0f5dce5  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9477559532676469}, {'name': 'Recall', 'type': 'recall', 'value': 0.955318646903623}, {'name': 'F1', 'type': 'f1', 'value': 0.9515222732614036}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9879370925335388}]}
To https://huggingface.co/keonju/korean_disease_ner
   0f5dce5..876235e  main -> main

   0f5dce5..876235e  main -> main



'https://huggingface.co/keonju/korean_disease_ner/commit/0f5dce51164df6c71678727d7986b8b4fff0385f'

In [58]:
trainer.save_model('/content/drive/MyDrive/sc2/ner_min')

Saving model checkpoint to /content/drive/MyDrive/sc2/ner_min
Configuration saved in /content/drive/MyDrive/sc2/ner_min/config.json
Model weights saved in /content/drive/MyDrive/sc2/ner_min/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/sc2/ner_min/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/sc2/ner_min/special_tokens_map.json
Saving model checkpoint to korean_disease_ner
Configuration saved in korean_disease_ner/config.json
Model weights saved in korean_disease_ner/pytorch_model.bin
tokenizer config file saved in korean_disease_ner/tokenizer_config.json
Special tokens file saved in korean_disease_ner/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9477559532676469}, {'name': 'Recall', 'type': 'recall', 'value': 0.955318646903623}, {'name': '